In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Scrapping Data

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print( tabulate(df[0], headers='keys', tablefmt='psql') )

+-----+---------------+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|     | Postal code   | Borough          | Neighborhood                                                                                                                                  |
|-----+---------------+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | M1A           | Not assigned     | nan                                                                                                                                           |
|   1 | M2A           | Not assigned     | nan                                                                                                                                           |
|   2 | M3A           | North York       | Parkwoods             

## Making Dataframe from raw Data

In [43]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [44]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [45]:
df.dtypes

Postal code     object
Borough         object
Neighborhood    object
dtype: object

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


## Splitting Data so that it is easily accessable

In [47]:
df["Neighborhood"]= df["Neighborhood"].str.split("/") 
  
# df display 
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


## droping not assigned Values

In [49]:
empty = 'Not assigned'
df = df[(df['Postal code'] != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [42]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


### rename column names, as it was disturbing

In [65]:
df.columns=['Postalcode','Borough','Neighborhood']

In [66]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


In [50]:
df.shape

(103, 3)

# Installing GeoCoder

In [52]:
!pip install geocoder

     |████████████████████████████████| 98 kB 2.0 MB/s eta 0:00:011


## But Unfortunately, It was not able to fetch records. so read from csv

In [59]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Again rename column Names

In [63]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [64]:
df_lon_lat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging Data so that final Dataframe Comes in

In [70]:
df_final = pd.merge(df,df_lon_lat[['Postalcode','Latitude', 'Longitude']],on='Postalcode')
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park , Harbourfront]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor , Lawrence Heights]",43.718518,-79.464763
4,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]",43.662301,-79.389494
